# W_GL_ACCOUNT_DS ETL Process
### Oracle EBS GL_CODE_COMBINATIONS to EDW W_GL_ACCOUNT_DS
#### Includes segment extraction and balancing/cost center/natural account logic

In [ ]:
%sql
-- 1) PARAMETERS
CREATE OR REPLACE TEMP VIEW etl_params AS
SELECT CASE WHEN COUNT(*) > 0 THEN 'Y' ELSE 'N' END AS IS_INCREMENTAL,
  COALESCE(date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'), '1970-01-01 00:00:00') AS LAST_EXTRACT_DATE,
  380 AS DATASOURCE_NUM_ID, '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
  '1' AS TENANT_ID, 2624523 AS ETL_PROC_WID, 12345 AS EXECUTION_ID, 30 AS PRUNE_DAYS
FROM workspace.oracle_edw.w_etl_load_dates
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION'
  AND DATASOURCE_NUM_ID = 380 AND ETL_USAGE_CODE = '__NOT_APPLICABLE__' AND COMMITTED = '1';

In [ ]:
%sql
-- 2) STAGING: Extract GL_CODE_COMBINATIONS with segment logic
-- NOTE: This is simplified - full implementation would include all 30 segments
CREATE OR REPLACE TEMP VIEW stg_gl_accounts AS
SELECT
  gcc.SEGMENT1, gcc.SEGMENT2, gcc.SEGMENT3, gcc.SEGMENT4, gcc.SEGMENT5,
  gcc.SEGMENT6, gcc.SEGMENT7, gcc.SEGMENT8, gcc.SEGMENT9, gcc.SEGMENT10,
  gcc.LAST_UPDATE_DATE, gcc.CHART_OF_ACCOUNTS_ID,
  gcc.ACCOUNT_TYPE, gcc.ENABLED_FLAG AS ACTIVE_FLAG,
  gcc.CONCATENATED_SEGMENTS AS CONCATENATED_SEGMENT_VALUE,
  -- Extract balancing segment based on structure definition
  CASE bs.BALANCING_SEGMENT_COLUMN
    WHEN 'SEGMENT1' THEN gcc.SEGMENT1
    WHEN 'SEGMENT2' THEN gcc.SEGMENT2
    WHEN 'SEGMENT3' THEN gcc.SEGMENT3
    ELSE NULL
  END AS BALANCING_SEGMENT_NUM,
  CAST(bs.FLEX_VALUE_SET_ID AS STRING) AS BALANCING_SEGMENT_ATTRIB,
  -- Extract cost center segment
  CASE cc.COST_CENTER_SEGMENT_COLUMN
    WHEN 'SEGMENT1' THEN gcc.SEGMENT1
    WHEN 'SEGMENT2' THEN gcc.SEGMENT2
    ELSE NULL
  END AS COST_CENTER_NUM,
  CAST(cc.FLEX_VALUE_SET_ID AS STRING) AS COST_CENTER_ATTRIB,
  -- Extract natural account segment
  CASE na.NATURAL_ACCOUNT_SEGMENT_COLUMN
    WHEN 'SEGMENT1' THEN gcc.SEGMENT1
    WHEN 'SEGMENT2' THEN gcc.SEGMENT2
    ELSE NULL
  END AS NATURAL_ACCOUNT_NUM,
  CAST(na.FLEX_VALUE_SET_ID AS STRING) AS NATURAL_ACCOUNT_ATTRIB,
  COALESCE(CAST(gcc.CHART_OF_ACCOUNTS_ID AS STRING), '__UNASSIGNED__') AS CHART_OF_ACCOUNTS,
  COALESCE(gcc.ACCOUNT_TYPE, '__UNASSIGNED__') AS RECON_TYPE_CODE,
  CAST(gcc.LAST_UPDATED_BY AS STRING) AS CREATED_BY_ID,
  CAST(gcc.LAST_UPDATED_BY AS STRING) AS CHANGED_BY_ID,
  CAST(gcc.CODE_COMBINATION_ID AS STRING) AS INTEGRATION_ID,
  gcc.LAST_UPDATE_DATE AS CHANGED_ON_DT,
  cc.LAST_UPDATE_DATE AS AUX1_CHANGED_ON_DT,
  bs.LAST_UPDATE_DATE AS AUX3_CHANGED_ON_DT
FROM workspace.oracle_ebs.GL_CODE_COMBINATIONS gcc
LEFT JOIN workspace.oracle_ebs.FND_ID_FLEX_STRUCTURES bs
  ON gcc.CHART_OF_ACCOUNTS_ID = bs.ID_FLEX_NUM
LEFT JOIN workspace.oracle_ebs.FND_SEGMENT_ATTRIBUTE_VALUES cc
  ON gcc.CHART_OF_ACCOUNTS_ID = cc.ID_FLEX_NUM
LEFT JOIN workspace.oracle_ebs.FND_SEGMENT_ATTRIBUTE_VALUES na
  ON gcc.CHART_OF_ACCOUNTS_ID = na.ID_FLEX_NUM
WHERE gcc.ENABLED_FLAG = 'Y';

In [ ]:
%sql
-- 3) INSERT INTO W_GL_ACCOUNT_DS
INSERT INTO workspace.oracle_edw.w_gl_account_ds
(
  SEGMENT1, SEGMENT2, SEGMENT3, SEGMENT4, SEGMENT5, SEGMENT6, SEGMENT7, SEGMENT8, SEGMENT9, SEGMENT10,
  CHART_OF_ACCOUNTS, CONCATENATED_SEGMENT_VALUE, ACTIVE_FLG,
  BALANCING_SEGMENT_NUM, BALANCING_SEGMENT_ATTRIB,
  COST_CENTER_NUM, COST_CENTER_ATTRIB,
  NATURAL_ACCOUNT_NUM, NATURAL_ACCOUNT_ATTRIB,
  RECON_TYPE_CODE, CREATED_BY_ID, CHANGED_BY_ID,
  CHANGED_ON_DT, AUX1_CHANGED_ON_DT, AUX3_CHANGED_ON_DT,
  DATASOURCE_NUM_ID, INTEGRATION_ID, TENANT_ID, DELETE_FLG
)
SELECT
  s.SEGMENT1, s.SEGMENT2, s.SEGMENT3, s.SEGMENT4, s.SEGMENT5, s.SEGMENT6, s.SEGMENT7, s.SEGMENT8, s.SEGMENT9, s.SEGMENT10,
  s.CHART_OF_ACCOUNTS, s.CONCATENATED_SEGMENT_VALUE, s.ACTIVE_FLAG,
  s.BALANCING_SEGMENT_NUM, s.BALANCING_SEGMENT_ATTRIB,
  s.COST_CENTER_NUM, s.COST_CENTER_ATTRIB,
  s.NATURAL_ACCOUNT_NUM, s.NATURAL_ACCOUNT_ATTRIB,
  s.RECON_TYPE_CODE, s.CREATED_BY_ID, s.CHANGED_BY_ID,
  s.CHANGED_ON_DT, s.AUX1_CHANGED_ON_DT, s.AUX3_CHANGED_ON_DT,
  p.DATASOURCE_NUM_ID, s.INTEGRATION_ID, p.TENANT_ID, 'N'
FROM stg_gl_accounts s
CROSS JOIN etl_params p;

In [ ]:
%sql
-- 4) UPDATE CONTROL AND LOG (Standard pattern)
MERGE INTO workspace.oracle_edw.w_etl_load_dates AS tgt
USING (
  SELECT p.DATASOURCE_NUM_ID, 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLACCOUNTDIMENSION' AS package_name,
    'W_GL_ACCOUNT_DS' AS target_table_name, p.ETL_USAGE_CODE, p.ETL_PROC_WID,
    p.EXECUTION_ID AS load_plan_id, current_timestamp() AS etl_load_date,
    CASE WHEN p.IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END AS committed,
    date_add(current_timestamp(), -p.PRUNE_DAYS) AS wip_load_start_date
  FROM etl_params p
) src
ON tgt.datasource_num_id = src.datasource_num_id AND tgt.package_name = src.package_name
WHEN MATCHED THEN UPDATE SET tgt.etl_load_date = src.etl_load_date
WHEN NOT MATCHED THEN INSERT VALUES (src.datasource_num_id, src.package_name, src.target_table_name,
  src.etl_usage_code, src.etl_proc_wid, src.load_plan_id, src.wip_load_start_date, NULL, src.etl_load_date, src.committed);

In [ ]:
%sql
DROP VIEW IF EXISTS stg_gl_accounts;
DROP VIEW IF EXISTS etl_params;